In [1]:
# Import Libraries
import pandas as pd 
import matplotlib.pyplot as plt
import os
from datetime import datetime
import time
%config IPCompleter.greedy=True

In [32]:
#02 READ IN DATA 

path = "https://data.ny.gov/api/views/3x8r-34rs/rows.csv?accessType=DOWNLOAD"
#path = "../Resources/Solar_Installation_Data Raw.csv"

solar = pd.read_csv(path)
solar.head()

,Reporting Period,Project Number,City,County,State,Zip Code,Sector,Program Type,Solicitation,Electric Utility,...,Total PV Module Quantity,Project Cost,$Incentive,Total Nameplate kW DC,Expected KWh Annual Production,Remote Net Metering,Affordable Solar,Community Distributed Generation,Green Jobs Green New York Participant,Location 1
0,05/31/2018,4260-88892,Three Mile Bay,Jefferson,NY,13693,Residential,Residential/Small Commercial,PON 2112,National Grid,...,32.0,29680.00,4240.0,8.80,10330.0,No,No,No,NaN,"Three Mile\nBay, NY 13693\n"
1,05/31/2018,5098-68726,North Babylon,Suffolk,NY,11703,Residential,Residential/Small Commercial,PON 2112LI,PSEG Long Island,...,25.0,20800.00,886.0,6.15,7219.0,No,No,No,NaN,"North Babylon, NY 11703\n(40.733915, -73.323802)"
2,05/31/2018,89350,Queens,Queens,NY,11434,Residential,Residential/Small Commercial,PON 2112,Consolidated Edison,...,21.0,30791.25,2898.0,7.25,8504.0,NaN,No,No,NaN,"Queens, NY 11434\n(40.676003, -73.775656)"
3,05/31/2018,93198,Bronx,Bronx,NY,10456,Residential,Residential/Small Commercial,PON 2112,Consolidated Edison,...,14.0,13906.00,1848.0,4.62,5423.0,NaN,No,No,NaN,"Bronx, NY 10456\n(40.830652, -73.908566)"
4,05/31/2018,94091,Brooklyn,Kings,NY,11220,Residential,Residential/Small Commercial,PON 2112,Consolidated Edison,...,84.0,64300.00,2117.0,5.29,6212.0,NaN,No,No,Residential,"Brooklyn, NY 11220\n(40.640771, -74.016133)"


In [34]:
#03 DATA CLEANING

#NARROW TO RELEVANT VARIABLES
solar = solar[["Project Number", "City", "Zip Code",
               "Sector", "Electric Utility", 
               "Purchase Type", "Project Status",
               "Date Application Received", "Date Completed",
               "Project Cost","$Incentive", "Total Nameplate kW DC", 
               "Expected KWh Annual Production", "Contractor", 
               "Primary PV Module Manufacturer", "Location 1" ]]

solar = solar.loc[solar["Project Status"] == "Complete"]
solar.dropna(how = 'any')

#Convert Date variables to datetime
solar['Date Completed'] = pd.to_datetime(solar['Date Completed'] )
solar['Date Application Received'] = pd.to_datetime(solar['Date Application Received'] )

#Convert datetimes to unix timestamps
def convert_time(time_dt):
    return int(time.mktime(time_dt.timetuple()))

#print columns with unix ts and length of project in days
solar['Date Application Received Int'] = solar.apply(lambda row: convert_time(row['Date Application Received']), axis=1)
solar['Date Completed Int'] = solar.apply(lambda row: convert_time(row['Date Completed']), axis=1)
solar['Project Days Int'] = solar.apply(lambda row: int((row['Date Completed Int']-row['Date Application Received Int'])/86400), axis=1)

#parse the datetime objects
solar['Completed Year'] = solar.apply(lambda row: row['Date Completed'].year, axis=1)
solar['Completed Month'] = solar.apply(lambda row: row['Date Completed'].month, axis=1)

#Create additional variables
solar['$_per_watt'] = round(solar['Project Cost'] / solar['Total Nameplate kW DC'] / 1000, 2)
solar['$_per_watt_wincentives'] = round((solar['Project Cost'] - solar['$Incentive'] ) / solar['Total Nameplate kW DC'] /1000, 2)


#Convert zip code to string for merge
#solar['Zip Code'] = solar['Zip Code'].to_string()
solar.head()


#grab data frame rows with a specific value df = df.loc[df.column == 'somevalue']
# Select from DataFrame using criteria from multiple columns
# (use `|` instead of `&` to do an OR)
#newdf = df[(df['column_one']>2004) & (df['column_two']==9)]



# Sort dataframe by multiple columns
#df = df.sort_values(['col1','col2','col3'],ascending=[1,1,0])

# Get top n for each group of columns in a sorted dataframe
# (make sure dataframe is sorted first)
#top5 = df.groupby(['groupingcol1', 'groupingcol2']).head(5)

,Project Number,City,Zip Code,Sector,Electric Utility,Purchase Type,Project Status,Date Application Received,Date Completed,Project Cost,...,Contractor,Primary PV Module Manufacturer,Location 1,Date Application Received Int,Date Completed Int,Project Days Int,Completed Year,Completed Month,$_per_watt,$_per_watt_wincentives
0,4260-88892,Three Mile Bay,13693,Residential,National Grid,Purchase,Complete,2016-02-01,2016-09-21,29680.00,...,"Fourth Coast, Inc",SolarWorld,"Three Mile\nBay, NY 13693\n",1454313600,1474441200,232,2016,9,3.37,2.89
2,89350,Queens,11434,Residential,Consolidated Edison,Lease,Complete,2017-05-30,2017-07-25,30791.25,...,SunPower Capital LLC,SunPower,"Queens, NY 11434\n(40.676003, -73.775656)",1496127600,1500966000,56,2017,7,4.25,3.85
3,93198,Bronx,10456,Residential,Consolidated Edison,Lease,Complete,2017-07-19,2017-09-20,13906.00,...,SunRun Inc.,LG Electronics,"Bronx, NY 10456\n(40.830652, -73.908566)",1500447600,1505890800,63,2017,9,3.01,2.61
7,2011-157041-SLPR,Old Field,11733,Residential,PSEG Long Island,Purchase,Complete,2005-10-09,2005-10-09,NaN,...,NaN,NaN,"Old Field, NY 11733\n(40.93232, -73.105)",1128841200,1128841200,0,2005,10,NaN,NaN
8,2011-158360-SLPR,Medford,11763,Residential,PSEG Long Island,Purchase,Complete,2009-04-23,2009-04-23,136952.00,...,NaN,NaN,"Medford, NY 11763\n(40.822436, -72.982416)",1240470000,1240470000,0,2009,4,13.36,9.95
